Problem is probably that <text> tag has to long content (full article)

check: 
    - https://stackoverflow.com/questions/33828728/what-is-the-maximum-size-of-an-xml-file-when-using-pythons-lxml-etree
    - https://stackoverflow.com/questions/11850345/using-python-lxml-etree-for-huge-xml-files
    - https://stackoverflow.com/questions/47229309/what-are-the-differences-between-lxml-and-elementtree

https://docs.python.org/3/library/xml.etree.elementtree.html#xml.etree.ElementTree.iterparse


Note

iterparse() only guarantees that it has seen the “>” character of a starting tag when it emits a “start” event, so the attributes are defined, but the contents of the text and tail attributes are undefined at that point. The same applies to the element children; they may or may not be present.

If you need a fully populated element, look for “end” events instead.


In [1]:
"""
Based on: "Processing Large XML Wikipedia Dumps that won't fit in RAM in Python without Spark" https://www.youtube.com/watch?v=AeRN4zI7Dhk

"""

import xml.etree.ElementTree as etree
import codecs
import csv
import time
import os
import re

PATH_WIKI_XML = 'F:/wikipedia-data/'
FILENAME_WIKI = 'enwiki-latest-pages-articles.xml'
FILENAME_ARTICLES = 'articlesDEBUG.csv'
FILENAME_REDIRECT = 'redirectDEBUG.csv'
FILENAME_CATEGORY = 'categoriesDEBUG.csv'
ENCODING = "utf-8"

In [2]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [3]:
def strip_tag_name(t):
    t = elem.tag
    idx = k = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t

In [4]:
pathWikiXML = os.path.join(PATH_WIKI_XML, FILENAME_WIKI)
pathArticles = os.path.join(PATH_WIKI_XML, FILENAME_ARTICLES)
# pathArticlesRedirect = os.path.join(PATH_WIKI_XML, FILENAME_REDIRECT)
pathCategory = os.path.join(PATH_WIKI_XML, FILENAME_CATEGORY)
pathRedirect = os.path.join(PATH_WIKI_XML, FILENAME_REDIRECT)

In [5]:
totalCount = 0
articleCount = 0
redirectCount = 0
categoryCount = 0
title = None
start_time = time.time()

In [6]:
with codecs.open(pathArticles, "w", ENCODING) as articlesFH:
    
    articlesWriter = csv.writer(articlesFH, quoting=csv.QUOTE_MINIMAL)
    #categoryWriter = csv.writer(categoryFH, quoting=csv.QUOTE_MINIMAL)
    #redirectWriter = csv.writer(redirectFH, quoting=csv.QUOTE_MINIMAL)

    articlesWriter.writerow(['id', 'title', 'parentCategories', 'links'])
    #categoryWriter.writerow(['id', 'title', 'parentCategories'])
    #redirectWriter.writerow(['id', 'title', 'redirect'])
    
    
    for event, elem in etree.iterparse(pathWikiXML, events=('start', 'end')):
        tname = strip_tag_name(elem.tag)

        if event == 'start':
            if tname == 'page':
                title = ''
                id = -1
                redirect = False
                inrevision = False
                isredirect = False
                ns = 0
                parentCategories = 'Not run'
                links = 'Not run'
                articleText = 'Category:xxxERROR IF IN DATAxxx'
            elif tname == 'revision':
                # Do not pick up on revision id's
                inrevision = True
            elif tname == 'title':
                    title = elem.text

            elif tname == 'id' and not inrevision and elem.text!=None:
                id = int(elem.text)
            elif tname == 'redirect':
                redirect = elem.get('title', '')
                isredirect = True
            elif tname == 'ns' and elem.text!=None:
                ns = int(elem.text)
            
            
        elif tname == 'text':
                articleText = str(elem.text)
                parentCategories = articleText[:20]
                #links = re.findall(r'\[\[(.*?)\]\]', articleText)
                       
        
        elif tname == 'page':
            totalCount += 1

            if ns == 14:
                categoryCount += 1
                #categoryWriter.writerow([id, title, parentCategories])
            elif isredirect:
                redirectCount += 1    
                #redirectWriter.writerow([id, title, redirect])
            elif ns == 0:
                articleCount += 1
                articlesWriter.writerow([id, title, parentCategories, links])
                
                # Category info is contained within <text> tag and in format [[Category:Autism| ]]
            
#             else:
#                 redirectCount += 1
#                 redirectWriter.writerow([id, title, redirect])

            if totalCount > 1 and (totalCount % 100000) == 0:
                print("{:,}".format(totalCount))

        elem.clear()
        
time_took = time.time() - start_time
print(f"Total runtime: {hms_string(time_took)}")

KeyboardInterrupt: 